Nome: Fabio Grassiotto  
RA: 890441

# Aula 9_10 - Implementar a ReAct  com LLaMa 3 70B (groq)

- Testar no dataset do IIRC - 50 primeiras perguntas com resposta (test_questions.json em anexo)  
- Usar o prompt do LLaMAIndex: https://github.com/run-llama/llama_index/blob/a87b63fce3cc3d24dc71ae170a8d431440025565/llama_index/agent/react/prompts.py  
- Salvar as respostas finais das 50 perguntas no JSON para exercício futuro de avaliação  
- Instruir o modelo a seguir a sequência Thougth, Action, Input, Observation (a observação não é do próprio modelo, mas resultado da busca)  
- É necessário usar o parâmetro stop_sequence="Observation:", para o modelo parar de gerar texto e esperar o retorno da busca. Implementem o código da busca e retornem os top-k documentos pro modelo (sugestão: k=5).  
- Instruir o modelo agir passo-a-passo (decomposição da pergunta).  
- Podem usar o LangChain, LLaMAindex ou outro framework. Ou implementar na mão.  
- Usar a busca como ferramenta  
- Usar o BM25 como buscador (repetir indexação do exercício passado)  
- Usar a indexação do Visconde: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipyn  


## Setup Environment
### Install packages

In [3]:
%%capture
%pip install -q torch
%pip install groq
%pip install llama-index-llms-groq
%pip install langchain
%pip install langchain-groq
%pip install -U sentence-transformers
%pip install faiss-cpu
%pip install rank_bm25
%pip install bs4
%pip install spacy
!python -m spacy download en_core_web_sm

### Imports

In [4]:
import os
import sys
import json
import torch
from collections import Counter
import string
import re
from bs4 import BeautifulSoup
from rank_bm25 import BM25Okapi
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm
import spacy
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd

import warnings
warnings.simplefilter('ignore')

### Collab setup

In [5]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_9_10"
    os.chdir(project_folder)
    !ls -la

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Globals

In [6]:
NUM_QUESTIONS = 50
LLM_MODEL_NAME="llama3-70b-8192"
SEARCH_MODEL_NAME = "sentence-transformers/msmarco-MiniLM-L-6-v3"

## IIRC Dataset

In [7]:
if not os.path.exists("dataset/context_articles.json"):
    !wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
    !tar -xzf context_articles.tar.gz --directory dataset
    !rm context_articles.tar.gz

In [8]:
test_dataset  = json.load(open('dataset/test_questions.json', 'r'))
articles = json.load(open('dataset/context_articles.json', 'r'))

In [9]:
test_dataset[0]

{'answer': {'type': 'span',
  'answer_spans': [{'text': 'sky and thunder god',
    'passage': 'zeus',
    'type': 'answer',
    'start': 83,
    'end': 102}]},
 'question': 'What is Zeus know for in Greek mythology?',
 'context': [{'text': 'he Palici the sons of Zeus',
   'passage': 'main',
   'indices': [684, 710]},
  {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
  {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
   'passage': 'Zeus',
   'indices': [0, 110]}],
 'question_links': ['Greek mythology', 'Zeus'],
 'title': 'Palici'}

### Load and parse

In [10]:
# Grab the first 50 questions  # type: ignorewith an answer
def grab_documents():

  questions_found = []
  num_questions_found = 0
  documents = []
  all_titles = []

  for item in test_dataset:
    question = item['question']
    answer = item['answer']
    answer_type = answer['type']

    if answer_type == 'binary' or answer_type == 'value':
      final_answer = answer['answer_value']
    elif answer_type == 'span':
      final_answer = answer['answer_spans'][0]['text']
    elif answer_type == 'none':
      final_answer = 'none'
    else:
      final_answer = 'An error perhaps, bad type'
      print(answer_type)

    if (final_answer == 'none'):
      # Skip this one.
      continue
    else:
      # Thats a good question.
      
      # First add some extra info in the context-part
      for context in item['context']:
        if context['passage'] == "main":
            # Cleanup html tags
            soup = BeautifulSoup(context['text'], 'html.parser')
            clean_text = soup.get_text()

            documents.append({
                    "title": item['title'].lower(),
                    "content": clean_text
              }
          )
        all_titles.append(item['title'].lower())

      # And then grab the text from the articles json 
      for link in item["question_links"]:
        if link.lower() in articles and link.lower() not in all_titles:
          # Cleanup html tags
          soup = BeautifulSoup(articles[link.lower()], 'html.parser')
          clean_text = soup.get_text()

          documents.append({
            "title": link,
            "content": clean_text
          })
        all_titles.append(link.lower())

      questions_found.append({"Question": question, "Answer": final_answer})
      num_questions_found += 1
      if (num_questions_found == NUM_QUESTIONS):
        # found our questions
        break

  return questions_found, documents, all_titles

questions_to_ask, documents, all_titles = grab_documents()

### Sequence segmentation (3 per document)

In [11]:
contents_list = []
nlp = spacy.load('en_core_web_sm')

for t in tqdm(documents):
    paragraph = t.get('content')
    paragraph = paragraph[:512] 

    sentences = list(sent.text for sent in nlp(paragraph).sents)
    sentences = sentences[:3]
    excerpt = ""
    for s in sentences:
        excerpt += s
    contents_list.append(excerpt)

100%|██████████| 124/124 [00:01<00:00, 115.18it/s]


### Inspecting data

In [12]:
len(questions_to_ask)

50

In [13]:
len(contents_list)

124

In [14]:
contents_list[:10]

['he Palici the sons of Zeus',
 'in Greek mythology',
 "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Greece and its civilization, and to gain understanding of the",
 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n',
 'he became aide-de-camp to King Victor Emm

In [15]:
all_titles[:10]

['palici',
 'palici',
 'palici',
 'greek mythology',
 'zeus',
 'giovanni messe',
 'giovanni messe',
 'world war i',
 'giovanni messe',
 'giovanni messe']

In [16]:
questions_to_ask[0]

{'Question': 'What is Zeus know for in Greek mythology?',
 'Answer': 'sky and thunder god'}

## FAISS Indexing

In [17]:
model = SentenceTransformer(SEARCH_MODEL_NAME)
model.to(device)

embeddings = model.encode(contents_list, show_progress_bar=True)

Batches: 100%|██████████| 4/4 [00:00<00:00,  5.26it/s]


In [18]:
# Create a FAISS index from the embeddings
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

## Langchain Groq API

In [19]:
def load_groq_key():
    try:
        # Open and read the entire content of the file
        with open("groq-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None
    
chat = ChatGroq(temperature=0, groq_api_key=load_groq_key(), model_name=LLM_MODEL_NAME)

In [20]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:\n\n1. **Interactive Systems**: In applications like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, creating a more natural and engaging user experience. This is particularly important in customer-facing applications where delayed responses can lead to frustration and abandonment.\n2. **Real-time Decision Making**: In areas like finance, healthcare, and cybersecurity, timely decision-making is critical. Low-latency LLMs can quickly analyze vast amounts of data, enabling swift identification of patterns, anomalies, and insights, which inform timely decisions.\n3. **Live Content Generation**: Low-latency LLMs are essential for generating content in real-time, such as live subtitles, closed captions, or automated summarization of

## ReAct Agent Implementation

### Prompt from Langchain (https://smith.langchain.com/hub/hwchase17/react-chat)

This prompt was modified:

- to remove the chat history, so every question is a new chat with the LLM.
- to request the model to use short answers, and to answer with numerals instead of words for numbers, to increase our F1 score.

In [21]:
LANGCHAIN_CHAT_SYSTEM_HEADER = """\

Assistant is a large language model trained by Meta AI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
Assistant has access to the following tools:
{tools}
To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```
Your final answer must be short, no more than 10 words, and use numerals instead of words for numbers.
Begin!
New input: {input}
{agent_scratchpad}
"""

### Agent tool definition

Top 5 and top 10 hits returned from the search.

In [51]:
@tool
def get_top5(question: str) -> list[str]:
    """Returns a list of five strings with texts to help solve a question."""
    xq = model.encode([question])
    _, I = index.search(xq, 5)  # search

    results = []
    for i in range(0, 5):
        results.append(contents_list[I[0][i]])    
    return results

@tool
def get_top10(question: str) -> list[str]:
    """Returns a list of ten strings with texts to help solve a question."""
    xq = model.encode([question])
    _, I = index.search(xq, 10)  # search

    results = []
    for i in range(0, 10):
        results.append(contents_list[I[0][i]])    
    return results

from sentence_transformers import CrossEncoder

@tool
def get_top5_rerank(question: str) -> list[str]:
    """Returns a list of five strings with texts to help solve a question, using a cross-encoder for re-ranking."""
    xq = model.encode([question])
    D, I = index.search(xq, 15)  # search
    results = []
    for i in range(0, 15):
        results.append(contents_list[I[0][i]]) 

    crossencoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    # Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    crossencoder_inputs = [[question, passage] for passage in results]
    scores = crossencoder.predict(crossencoder_inputs)

    # Sort the scores in decreasing order
    reranked_results = [{"input": inp, "score": score} for inp, score in zip(crossencoder_inputs, scores)]
    reranked_results = sorted(reranked_results, key=lambda x: x["score"], reverse=True)

    cross_enc_res = []
    for res in reranked_results:
        cross_enc_res.append(res['input'][1])
    cross_enc_res = cross_enc_res[:5]
    return cross_enc_res

@tool
def get_top10_rerank(question: str) -> list[str]:
    """Returns a list of five strings with texts to help solve a question, using a cross-encoder for re-ranking."""
    xq = model.encode([question])
    D, I = index.search(xq, 30)  # search
    results = []
    for i in range(0, 30):
        results.append(contents_list[I[0][i]]) 

    crossencoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    # Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    crossencoder_inputs = [[question, passage] for passage in results]
    scores = crossencoder.predict(crossencoder_inputs)

    # Sort the scores in decreasing order
    reranked_results = [{"input": inp, "score": score} for inp, score in zip(crossencoder_inputs, scores)]
    reranked_results = sorted(reranked_results, key=lambda x: x["score"], reverse=True)

    cross_enc_res = []
    for res in reranked_results:
        cross_enc_res.append(res['input'][1])
    cross_enc_res = cross_enc_res[:5]
    return cross_enc_res

## Execution Run

### Evaluation functions

In [25]:
def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

### Analysing the output for 50 questions

In [26]:
def evaluate(executor):
    # Create dataframe with question, answer, Llm answer and scores
    df_results = pd.DataFrame(columns=['question', 'answer', 'LLM answer', 'F1', 'Exact Match'])

    for item in tqdm(questions_to_ask):
        question = item.get('Question')
        answer = normalize_answer(item.get('Answer'))

        # RAG context using search
        agent_answer = executor.invoke({"input": "Question: " + question})
        llm_answer = normalize_answer(agent_answer['output'])
        f1_score = token_f1_score(llm_answer, answer)
        e_match_score = 1 if llm_answer == answer else 0
        
        row = pd.Series([question, answer, llm_answer, f1_score, e_match_score], index=df_results.columns)
        df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)

    return df_results

In [54]:
def run_agent(tool, results_file):
    prompt = PromptTemplate.from_template(LANGCHAIN_CHAT_SYSTEM_HEADER)
    llm = ChatGroq(temperature=0,
                model_name=LLM_MODEL_NAME,
                api_key=load_groq_key())
    tools = [tool]
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
    df = evaluate(agent_executor)

    print(df)
    df.to_csv(results_file)
    print()
    print("Metrics:")
    print(f"F1 score: Avg: {df['F1'].mean():.2f}. Std: {df['F1'].std():.2f}")
    print(f"Exact Match score: Avg: {df['Exact Match'].mean():.2f}. Std: {df['Exact Match'].std():.2f}")

### Searching with Tool returning top 5 hits

In [55]:
run_agent(get_top5, "results/react_top5.csv")

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', 'in Greek mythology', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions o

  2%|▏         | 1/50 [00:02<01:40,  2.05s/it]

Thought: Do I need to use a tool? No
Final Answer: King of gods on Mount Olympus.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'The 1936 Summer Olympics (German: Olympische Sommerspiele 1936), officially known as the Games of the XI Olympiad, was an international multi-sport event held in 1936 in Berlin, Nazi Germany.Berlin won the bid to host the Games over Barcelona, Spain, on 26 April 1931, at the 29th IOC Session in Barcelona.It marked the second and final time the International Olympic Committee gathered to vote in a city that was bidding to host those Games.\n\n', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European W

  4%|▍         | 2/50 [00:04<01:53,  2.36s/it]

...

Do I need to use a tool? No
Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'The Battle of Kulm was a battle near the town Kulm (Chlumec) and the v

 98%|█████████▊| 49/50 [09:21<00:13, 13.56s/it]

Do I need to use a tool? No
Final Answer: Fort Donelson had higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'In the Battle of Bautzen (20–2

100%|██████████| 50/50 [09:34<00:00, 11.49s/it]

Do I need to use a tool? No
Final Answer: United States

> Finished chain.
                                             question  \
0           What is Zeus know for in Greek mythology?   
1   How long had the First World War been over whe...   
2   How old was Messe when the First World War sta...   
3   How long had Angela Scoular been acting profes...   
4   What is the capacity of the stadium where Brun...   
5   Which stadium where Brunt played can hold more...   
6           In which country was Wilhelm Müller born?   
7   Which battle Wilhelm Müller fought in while in...   
8   How much time had passed between Wilhelm's ret...   
9   Which of Wilhelm's direct male descendants liv...   
10  In which Italian region did Pesce studied medi...   
11    Which country hosted the 1948 Leprosy Congress?   
12  When was the sports organization that the Turk...   
13  What was the founding date of the championship...   
14  How many years did the player who  top-scored ...   
15           

### Searching with Tool returning top 5 hits with Rerank

In [56]:
run_agent(get_top5_rerank, "results/react_top5_rerank.csv")

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Greec

  2%|▏         | 1/50 [00:04<03:34,  4.39s/it]

Do I need to use a tool? No
Final Answer: King of gods in Greek mythology

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918.Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70\xa0million military personnel, including 60 million Europeans, making it one of the largest wars in history.It is also one of the deadliest conflicts', 'Messe was born in Mesagne, in the Province of Brindisi in the Apulia region of Italy on 10 December 1883

  4%|▍         | 2/50 [00:09<04:03,  5.06s/it]

...

Do I need to use a tool? No
Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', 'The Battle of Fort Henry was fought on February 6, 1862, in Donelson Middle Tennessee, during the American Civil War.It was the first important victory for the Union and Brig.Gen. Ulysses S. Grant in the Western Theater.\n\n', "The Battle of Fort Donelson was fought from February 11–16, 1862, in the Western Theater of the American Civil War.The Union capture of the Confederate fort near the Tennessee–Kentucky border opened the Cumberland River, an important avenue for the invasion of the South.The Union's success also elevated Brig.", "In the Battle of Lützen (German: Sc

 98%|█████████▊| 49/50 [09:56<00:13, 13.91s/it]

Do I need to use a tool? No
Final Answer: Fort Donelson had higher casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5_rerank
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', 'El Salvador (; ), officially the Republic of El Salvador (, literally "Republic of The Savior"), is the smallest and the most densely populated country in Central America.It is bordered on the northeast by Honduras, on the northwest by Guatemala, and on the south by the Pacific Ocean.El Salvador\'s capital and largest city is San Salvador.', 'Joya de Cerén (Jewel of Cerén in the Spanish language) is an archaeological site in La Libertad Department, El Salvador, featuring a pre-Columbian Maya farming village.\nThe ancient Maya site of Joy

100%|██████████| 50/50 [10:10<00:00, 12.21s/it]

Do I need to use a tool? No
Final Answer: United States

> Finished chain.
                                             question  \
0           What is Zeus know for in Greek mythology?   
1   How long had the First World War been over whe...   
2   How old was Messe when the First World War sta...   
3   How long had Angela Scoular been acting profes...   
4   What is the capacity of the stadium where Brun...   
5   Which stadium where Brunt played can hold more...   
6           In which country was Wilhelm Müller born?   
7   Which battle Wilhelm Müller fought in while in...   
8   How much time had passed between Wilhelm's ret...   
9   Which of Wilhelm's direct male descendants liv...   
10  In which Italian region did Pesce studied medi...   
11    Which country hosted the 1948 Leprosy Congress?   
12  When was the sports organization that the Turk...   
13  What was the founding date of the championship...   
14  How many years did the player who  top-scored ...   
15           

### Searching with Tool returning top 10 hits

In [58]:
run_agent(get_top10, "results/react_top10.csv")

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...


Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', 'in Greek mythology', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Greece and its civilization, 

  2%|▏         | 1/50 [00:15<12:37, 15.45s/it]

Do I need to use a tool? No
Final Answer: King of gods in Greek mythology

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'The 1936 Summer Olympics (German: Olympische Sommerspiele 1936), officially known as the Games of the XI Olympiad, was an international multi-sport event held in 1936 in Berlin, Nazi Germany.Berlin won the bid to host the Games over Barcelona, Spain, on 26 April 1931, at the 29th IOC Session in Barcelona.It marked the second and final time the International Olympic Committee gathered to vote in a city that was bidding to host those Games.\n\n', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was

...

Thought: Do I need to use a tool? No
Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'The Battle of Kulm was a battle near the town Kulm (Chlumec)

 98%|█████████▊| 49/50 [12:47<00:16, 16.59s/it]

Do I need to use a tool? No
Final Answer: Fort Donelson had higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', "In the Battle of Lützen (German: Schlacht von Großgörschen, 2 May 1813), Napoleon I of France halted the advances of the Sixth Coalition after the French invasion of Russia and the massive French losses in the campaign.The Russian commander, Prince Peter Wittgenstein, attempting to forestall Napoleon's capture of Leipzig, attacked the isolated French right wing near Lützen, Germany.After a day of heavy fighting, the combined Prussian and Russian force retreated; due to French losses and a shortage of Fren", 'In the Battle of Bautzen (20–

100%|██████████| 50/50 [13:05<00:00, 15.70s/it]

Do I need to use a tool? No
Final Answer: United States

> Finished chain.
                                             question  \
0           What is Zeus know for in Greek mythology?   
1   How long had the First World War been over whe...   
2   How old was Messe when the First World War sta...   
3   How long had Angela Scoular been acting profes...   
4   What is the capacity of the stadium where Brun...   
5   Which stadium where Brunt played can hold more...   
6           In which country was Wilhelm Müller born?   
7   Which battle Wilhelm Müller fought in while in...   
8   How much time had passed between Wilhelm's ret...   
9   Which of Wilhelm's direct male descendants liv...   
10  In which Italian region did Pesce studied medi...   
11    Which country hosted the 1948 Leprosy Congress?   
12  When was the sports organization that the Turk...   
13  What was the founding date of the championship...   
14  How many years did the player who  top-scored ...   
15           

### Searching with Tool returning top 10 hits with Rerank

In [57]:
run_agent(get_top10_rerank, "results/react_top10_rerank.csv")

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10_rerank
Action Input: What is Zeus know for in Greek mythology?['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus.His name is cognate with the first element of his Roman equivalent Jupiter.His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\n', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore.These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Gree

  2%|▏         | 1/50 [00:12<10:13, 12.53s/it]

Do I need to use a tool? No
Final Answer: King of gods in Greek mythology

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10_rerank
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918.Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70\xa0million military personnel, including 60 million Europeans, making it one of the largest wars in history.It is also one of the deadliest conflicts', 'Messe was born in Mesagne, in the Province of Brindisi in the Apulia region of Italy on 10 December 188

  4%|▍         | 2/50 [00:28<11:28, 14.34s/it]

...

Do I need to use a tool? No
Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10_rerank
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', 'The Battle of Fort Henry was fought on February 6, 1862, in Donelson Middle Tennessee, during the American Civil War.It was the first important victory for the Union and Brig.Gen. Ulysses S. Grant in the Western Theater.\n\n', "The Battle of Fort Donelson was fought from February 11–16, 1862, in the Western Theater of the American Civil War.The Union capture of the Confederate fort near the Tennessee–Kentucky border opened the Cumberland River, an important avenue for the invasion of the South.The Union's success also elevated Brig.", "In the Battle of Lützen (German: S

 98%|█████████▊| 49/50 [10:22<00:12, 12.95s/it]

Do I need to use a tool? No
Final Answer: Fort Donelson had higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top10_rerank
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', 'El Salvador (; ), officially the Republic of El Salvador (, literally "Republic of The Savior"), is the smallest and the most densely populated country in Central America.It is bordered on the northeast by Honduras, on the northwest by Guatemala, and on the south by the Pacific Ocean.El Salvador\'s capital and largest city is San Salvador.', 'The 1948 World Leprosy Congress in Havana endorsed the Latin-American strategy', 'Costa Rica (, ; ; literally "Rich Coast"), officially the Republic of Costa Rica (), is a sovereign state in 

100%|██████████| 50/50 [10:36<00:00, 12.73s/it]

Thought: Do I need to use a tool? No
Final Answer: El Salvador

> Finished chain.
                                             question  \
0           What is Zeus know for in Greek mythology?   
1   How long had the First World War been over whe...   
2   How old was Messe when the First World War sta...   
3   How long had Angela Scoular been acting profes...   
4   What is the capacity of the stadium where Brun...   
5   Which stadium where Brunt played can hold more...   
6           In which country was Wilhelm Müller born?   
7   Which battle Wilhelm Müller fought in while in...   
8   How much time had passed between Wilhelm's ret...   
9   Which of Wilhelm's direct male descendants liv...   
10  In which Italian region did Pesce studied medi...   
11    Which country hosted the 1948 Leprosy Congress?   
12  When was the sports organization that the Turk...   
13  What was the founding date of the championship...   
14  How many years did the player who  top-scored ...   
15    